## Notebook de pré-processamento da planilha BD Acidentes.

### Imports e dados

In [ ]:
import datetime
import json
import os
import unicodedata

import numpy as np
import pandas as pd
from thefuzz import fuzz, process
from unidecode import unidecode

In [ ]:
from resolve_path import ajuste_path, read_input

In [ ]:
# Se nao tiver a pasta dados, criar a pasta dados e colocar o arquivo bd acidentes mais recente dentro dela
path = "data/"

pathIn = path + "input/"
pathUtil = path + "util/"
pathUtil = ajuste_path(pathUtil)

arquivo = "BD_Acidentes_13_08.csv"

df = read_input(pathIn + arquivo)

df.info()

In [ ]:
# abrindo dataframe das coordenadas e seus locais de instalação correspondentes
dataFrameInstalacao = pd.read_csv(
    pathUtil + "local_coordenada.csv", sep=";")
dataFrameInstalacao

### Definição de Funções

In [ ]:
# Printa os tipos de dados presentes na coluna


def checa_tipos(coluna):
    count_string = 0
    count_float = 0
    count_int = 0
    count_na = 0
    count_outros = 0
    for i in coluna:
        if pd.isna(i):
            count_na += 1
        elif type(i) == str:
            count_string += 1
        elif type(i) == float:
            count_float += 1
        elif type(i) == int:
            count_int += 1
        else:
            count_outros += 1
    print("NAs: ", count_na)
    print("Strs: ", count_string)
    print("Floats: ", count_float)
    print("Ints: ", count_int)
    print("Outros: ", count_outros)


# Exibe informações sobre a coluna
def exibe_coluna(nome_col):
    print(checa_tipos(df[nome_col]))
    print()
    qt = df[nome_col].nunique()
    print("quantidade de diferentes entradas:", qt)
    print()
    if qt <= 10:
        print("Os diferentes valores nas colunas:", df[nome_col].unique())
        print()
    print(df[nome_col].value_counts(dropna=False))
    print()
    return


# Converte datas do excel para datetime
def converte_data_excel(value):
    excel_data_inicio = datetime.datetime(1899, 12, 30)
    if isinstance(value, int):
        return excel_data_inicio + datetime.timedelta(days=value)
    else:
        return value


# Conversão para Minúsculas e Remoção de Acentos
def formata_string(string):
    if isinstance(string, str):
        string = string.lower().strip()
        string = unidecode(string)
        if string == "-":
            return np.nan
        return string
    return string


# Remove os espaços das strings de uma coluna
def remove_espaços(string):
    r = string
    if type(string) == str:
        r = string.replace(" ", "")
    return r


# Remove coluna
def remove_coluna(df, coluna):
    if coluna in df.columns:
        df.drop(coluna, axis=1, inplace=True)
    return df


# Calcula distancias entre dois pontos de uma esfera a partir de suas latitudes e longitudes
def haversine(lat1, lon1, lat2, lon2):
    import math

    R = 6371

    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat / 2) * math.sin(d_lat / 2) + math.cos(
        math.radians(lat1)
    ) * math.cos(math.radians(lat2)) * math.sin(d_lon / 2) * math.sin(d_lon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d


# Define o Local de Instalação daquele acidente com base em suas coordenadas
def calcula_local_instalacao_acidente(raio, dfAcidentes, dfInstalacaoCoordenadas):
    lat_long = dfAcidentes[["Latitude", "Longitude"]].dropna()
    lat_long["Longitude"] = lat_long[
        "Longitude"
    ].apply(  # coordenada digitada errada (numero muito grande)
        lambda x: x / 100000 if x < -100 else x
    )
    lat_long["Longitude"] = lat_long[
        "Longitude"
    ].apply(  # sinal das coordenadas trocado
        lambda x: x * (-1) if x > 0 else x
    )

    lat_long["Longitude"] = lat_long[
        "Longitude"
    ].apply(  # virgulas das coordenadas digitadas errado
        lambda x: x * 10 if x > -10 else x
    )
    # pontos na antartica desconsiderados
    lat_long = lat_long[lat_long["Latitude"] > -40]

    for index, row in lat_long.iterrows():
        menor_distancia = raio
        idx_menor_distancia = -1
        tolerancia = 0.1
        df_locais_proximos = dfInstalacaoCoordenadas[(dfInstalacaoCoordenadas['Latitude'] < row['Latitude'] + tolerancia) & (dfInstalacaoCoordenadas['Latitude'] > row['Latitude'] - tolerancia) & (
            dfInstalacaoCoordenadas['Longitude'] < row['Longitude'] + tolerancia) & (dfInstalacaoCoordenadas['Longitude'] > row['Longitude'] - tolerancia)]
        for index2, row2 in df_locais_proximos.iterrows():
            distance = haversine(
                row2["Latitude"], row2["Longitude"], row["Latitude"], row["Longitude"]
            )
            if distance < menor_distancia:
                menor_distancia = distance
                idx_menor_distancia = index2
        if idx_menor_distancia == -1:
            continue
        dfAcidentes.loc[index, "Local de instalação"] = dfInstalacaoCoordenadas.loc[
            idx_menor_distancia, "Local de instalação"
        ]
        # colocar coluna distancia para a subestação mais proxima
        dfAcidentes.loc[index, "Distancia"] = menor_distancia

    # print("Quantidade de Acidentes:", qtdAcidentes)
    return dfAcidentes


# Função para encontrar valores similares
def achar_valores_similares(column, threshold=90):
    unique_values = column.unique()
    similar_pairs = []

    for value in unique_values:
        matches = process.extract(
            value, unique_values, limit=len(unique_values))
        for match in matches:
            if match[1] >= threshold and match[0] != value:
                similar_pairs.append((value, match[0], match[1]))

    return similar_pairs


def processa_coordenada(value):
    if isinstance(value, str):
        value = value.replace(",", ".")  # Substitui vírgula por ponto
        value = value.replace("°", "")  # Remove "°"
        return float(value)
    return value


# Inverte uma lista de dicionários onde os valores são listas de chaves a serem substituídas pelo valor correspondente.
def inverte_lista_de_dict(d):
    subs = {}
    for dic in d:
        if dic == {}:
            continue
        for valor, a_substituir in dic.items():

            for chave in a_substituir:
                subs[chave] = valor
    return subs


# Verifica se foi definida algum local de instalação fora da Eletrosul
def checar_acidente_fora_eletrosul(df):
    count_nao_vazios = 0
    # Filtrar o DataFrame com base na condição
    df_filtrada = df[df["Empresa"] != "cgt eletrosul"]
    df_filtrada = df[df["Empresa"] != "cgt eletrosul"]

    # Verificar se há valores não vazios na coluna_verificar
    valores_nao_vazios = df_filtrada["Local de instalação"].apply(
        lambda x: not pd.isnull(x) and (isinstance(x, str) and x.strip() != "")
    )

    # Retornar True se houver algum valor não vazio, caso contrário, False
    if valores_nao_vazios.any():
        count_nao_vazios += 1
    return count_nao_vazios


# Verificando se pode colocar | para separar o csv sem perda de valores
def possui_pipe(df):
    for column in df.columns:
        if df[column].apply(lambda x: "|" in str(x)).any():
            return True
    return False

## Coluna Codigo

Considerações: Foi removida esta coluna; todos os valores são 1

In [ ]:
remove_coluna(df, "Codigo")

## Coluna Empregado

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Empregado"] = df["Empregado"].apply(formata_string)
df["Empregado"]

## Coluna Classificação

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Classificação"] = df["Classificação"].apply(formata_string)
df["Classificação"] = df["Classificação"].apply(formata_string)
df["Classificação"]

## Coluna Nexo Causal

Considerações: Remoção de acentos e formatação para letra minúscula após transformar tudo em string

In [ ]:
df["Nexo causal"] = df["Nexo causal"].astype(dtype="string")

df["Nexo causal"] = df["Nexo causal"].apply(formata_string)

## Coluna Empresa

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Empresa"] = df["Empresa"].apply(formata_string)

## Coluna Fornecedor

Considerações: foi feita uma análise de alguns nomes parecidos para que se pudesse reduzir a quantidade de entradas únicas nesta coluna

In [ ]:
# Remove NA e strings vazias
df = df.dropna(subset=["Fornecedor"])

# Converte a coluna "Fornecedor" para o tipo string
df.loc[:, "Fornecedor"] = df["Fornecedor"].astype(dtype="string")

# Aplica a função formata_string na coluna "Fornecedor"
df.loc[:, "Fornecedor"] = df["Fornecedor"].apply(formata_string)
df.loc[:, "Fornecedor"] = df["Fornecedor"].apply(formata_string)

df.loc[df["Fornecedor"] == 'n.a.', "Fornecedor"] = np.nan

In [ ]:
df2022_forn = df[(df["Ano"] == 2022) & (df["Mês"] >= 11)]
df2023e2024_forn = df[(df["Ano"] > 2022)]

df_ultimos_18meses_forn = pd.concat(
    [df2022_forn, df2023e2024_forn], ignore_index=True)

df2022_antes_forn = df[(df["Ano"] == 2022) & (df["Mês"] < 11)]
df2020e2021_antes_forn = df[(df["Ano"] <= 2021)]

# O DataFrame "df_antes_ultimos_18meses" contém os registros relativos ao período anterior dos últimos 18 meses
df_antes_ultimos_18meses_forn = pd.concat(
    [df2020e2021_antes_forn, df2022_antes_forn], ignore_index=True
)

# Últimos 18 meses
lista_nomes_unicos_ultimos18meses_forn = df_ultimos_18meses_forn["Fornecedor"].unique(
)


# Meses anteriores
lista_nomes_unicos_antes_ultimos18meses_forn = df_antes_ultimos_18meses_forn[
    "Fornecedor"
].unique()

# Criação da lista que contém os dicionários
lista_nomes_empregos_forn = list()

for nome_emprego in lista_nomes_unicos_ultimos18meses_forn:
    # Condição que pula o registro, caso ele seja um Nan
    if (
        type(nome_emprego) == pd._libs.missing.NAType
    ):  # pd._libs.missing.NAType é um tipo de Nan do pandas
        continue

    else:
        # Criação da lista que vai conter os nomes a serem substituidos
        lista_nomes = list()

        # Adicionar dicionário à lista de dicionários
        # O índice é o valor a substituir que vem de lista_nomes_unicos_ultimos18meses
        # lista_nomes é lista vazia que irá conter os nomes a substituir que serão adicionados a lista no loop abaixo
        lista_nomes_empregos_forn.append({nome_emprego: lista_nomes})

        for index in range(len(lista_nomes_unicos_antes_ultimos18meses_forn)):
            if (
                fuzz.token_sort_ratio(
                    nome_emprego, lista_nomes_unicos_antes_ultimos18meses_forn[index]
                )
                > 80
            ):
                # print(nome_emprego + " - " + lista_nomes_unicos_antes_ultimos18meses[index])
                lista_nomes.append(
                    lista_nomes_unicos_antes_ultimos18meses_forn[index])

lista_nomes_empregos_forn

Através de uma análise visual da lista "lista_nomes_empregos_forn", percebe-se que as empresas estão, em sua maior parte, com nomes corretos.

Abaixo são feitas as substituições para nomes que apresentaram erros:

In [ ]:
df["Fornecedor"].replace(
    to_replace="vividense linhas de transmissao ltda.",
    value="vividence linhas de transmissao ltda",
    inplace=True,
)
df["Fornecedor"].replace(
    to_replace="ts infraestrutura e engenharia",
    value="ts infraestrutura e engenharia s.a",
    inplace=True,
)

# Nesse caso, acho que a pessoa quis escrever "eireli" e não "eirelli"
df["Fornecedor"].replace(
    to_replace="rp manutencao industrial eirelli",
    value="rp manutencao industrial",
    inplace=True,
)

## Coluna Nº do Contrato

Considerações: não foi feita alteração

## Coluna Segmento

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Segmento"] = df["Segmento"].astype(dtype="string")

df["Segmento"] = df["Segmento"].apply(formata_string)

## Coluna Função

Considerações: foi feita uma análise de alguns nomes parecidos para que se pudesse reduzir a quantidade de entradas únicas nesta coluna

In [ ]:
df["Função"] = df["Função"].astype(dtype="string")

df["Função"] = df["Função"].apply(formata_string)

A coluna "Função" do dataset original continha multiplos nomes para denotar o mesmo cargo (ex.: as strings "Tec. Elet.", "Tec Elet" e "Tecnico eletr." usadas para representar "Tecnico eletricista").
A fim de padronizar a nomenclatura utilizada para nomear cada função (emprego) dos trabalhadores acidentados, optou-se por utilizar as entradas únicas de string da coluna "Função" dos últimos 18 meses e procurar strings coincidentes na parcela restante do dataset. Tal escolha foi motivada pela informação passada pelo Clebsow na primeira reunião feita com ele, onde foi citado que os dados dos últimos 18 meses eram mais confiáveis.

Para realizar a padronização, escolheu-se utilizar a biblioteca **thefuzz** (https://github.com/seatgeek/thefuzz), a qual utiliza a distância de Levenshtein para realizar operações de string matching. 

A função utilizada para realizar a padronização não foi escolhida usando uma justificativa racional. A função "token_sort_ratio" foi escolhida por ter retornado os resultados mais coerentes (para 80% de similariedade entre as strings) entre as funções testadas da biblioteca thefuzz.

### Seleção do dataset dos últimos 18 meses

In [ ]:
df2022 = df[(df["Ano"] == 2022) & (df["Mês"] >= 11)]
df2023e2024 = df[(df["Ano"] > 2022)]

# O DataFrame "df_ultimos_18meses" contém os registros relativos aos últimos 18 meses
df_ultimos_18meses = pd.concat([df2022, df2023e2024], ignore_index=True)

### Seleção do dataset do período anterior aos últimos 18 meses

In [ ]:
df2022_antes = df[(df["Ano"] == 2022) & (df["Mês"] < 11)]
df2020e2021_antes = df[(df["Ano"] <= 2021)]

# O DataFrame "df_antes_ultimos_18meses" contém os registros relativos ao período anterior dos últimos 18 meses
df_antes_ultimos_18meses = pd.concat(
    [df2020e2021_antes, df2022_antes], ignore_index=True
)

df_antes_ultimos_18meses["Ano"].unique()

### Listas com nomes de empregos dos últimos 18 meses e dos meses anteriores

In [ ]:
# Últimos 18 meses
lista_nomes_unicos_ultimos18meses = df_ultimos_18meses["Função"].unique()


# Meses anteriores
lista_nomes_unicos_antes_ultimos18meses = df_antes_ultimos_18meses["Função"].unique(
)

### Seleção de nomes parecidos

As linhas abaixo realizam o processo de string matching. A lista "lista_nomes_empregos" para conter um conjunto de dicionários que conterão a string que irá substituir strings parecidas (a chave) e uma lista de strings a ser substituidas (o valor).

In [ ]:
# Criação da lista que contém os dicionários
lista_nomes_empregos = list()

for nome_emprego in lista_nomes_unicos_ultimos18meses:
    # Condição que pula o registro, caso ele seja um Nan
    if (
        type(nome_emprego) == pd._libs.missing.NAType
    ):  # pd._libs.missing.NAType é um tipo de Nan do pandas
        continue

    else:
        # Criação da lista que vai conter os nomes a serem substituidos
        lista_nomes = list()

        # Adicionar dicionário à lista de dicionários
        # O índice é o valor a substituir que vem de lista_nomes_unicos_ultimos18meses
        # lista_nomes é lista vazia que irá conter os nomes a substituir que serão adicionados a lista no loop abaixo
        lista_nomes_empregos.append({nome_emprego: lista_nomes})

        for index in range(len(lista_nomes_unicos_antes_ultimos18meses)):
            if (
                fuzz.token_sort_ratio(
                    nome_emprego, lista_nomes_unicos_antes_ultimos18meses[index]
                )
                > 80
            ):
                # print(nome_emprego + " - " + lista_nomes_unicos_antes_ultimos18meses[index])
                lista_nomes.append(
                    lista_nomes_unicos_antes_ultimos18meses[index])


len(lista_nomes_empregos)

O procedimento de string matching acabou retornando algumas relações que considerei erradas. Portanto, abaixo criei uma lista ("lista_indices errados") com as chaves dos dicionários contendo listas de strings que não eram parecidas com a string do seu respectivo dicionário.

In [ ]:
lista_indices_errados = [
    "tec eletricista",
    "montador",
    "tec. manutencao mecanica usina",
    "as. administrativo",
    "tec. eletricista",
    "tecnico manutencao de equipamentos",
    "eletricista",
    "eletricista de lt",
    "servicos gerais",
    "tecnico manutencao eletronica",
    "eletricista manutencao",
    "auxiliar tecnico de manutencao",
    "montador ii",
    "operador de trator",
    "operador de se",
    "operador de ute",
    "tecnico de manutencao eletromecanica",
    "tecnico de manutencao",
    "eletricista de manutencao",
    "eletricista de linhas",
    "tecnico em manutencao eletroeletronica",
    "tecnico em eletroeletronica",
    "auxiliar de manutencao",
    "eletricista de linha de transmissao",
    "as administrativo",
    "montador eletrico",
    "mecanico de maquinas pesadas",
    "auxiliar tecnico",
    "apoio tecnico/administrativo",
    "auxiliar de remocao",
    "apoio tecnico/administrativo",
    "eletricista manutencao",
    "tec eletricista lt",
    "tecnico eletricista",
    "servento de obras",
    "operador de subestacao",
    "montador i",
    "contadora",
    "ajudante de servicos gerais",
]

# As linhas abaixo tornam lista_nomes_empregos formatado corretamente para ser passado para a função .replace de pandas
for dicts in lista_nomes_empregos:
    for key in lista_indices_errados:
        if key in dicts:
            dicts.pop(key)

### Remoção dos dicionários com listas vazias e apropriação do dicionário de substituições

In [ ]:
lista_nomes_empregos_limpos = [
    d
    for d in lista_nomes_empregos
    if not any(isinstance(v, list) and not v for v in d.values())
]

lista_nomes_empregos_limpos = inverte_lista_de_dict(
    lista_nomes_empregos_limpos)

### Substituição dos nomes parecidos

In [ ]:
df["Função"].replace(lista_nomes_empregos_limpos, inplace=True)
df["Função"].replace(to_replace="mototrista", value="motorista", inplace=True)

# Substituição de nomes usados para denominar Nan para um valor Nan do pandas
df["Função"].replace(
    to_replace="n.a.", value=pd._libs.missing.NAType, inplace=True)
df["Função"].replace(
    to_replace="-", value=pd._libs.missing.NAType, inplace=True)
df["Função"].replace(
    to_replace="--", value=pd._libs.missing.NAType, inplace=True)

In [ ]:
len(lista_nomes_empregos)

## Coluna Sexo

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Sexo"] = df["Sexo"].astype(dtype="string")

df["Sexo"] = df["Sexo"].apply(formata_string)
df["Sexo"]

## Coluna Idade

Considerações: foi substituida a entrada diferente do padrão por algo que o obedeça, permitindo uma troca do tipo da coluna.
Foi substituido o cluster >60 para 61, para que todos os valores da coluna fossem numericos.

In [ ]:
print(df["Idade"].dtypes)

df["Idade"].replace(to_replace="> 60", value=61, inplace=True)

df["Idade"].dtypes

## Coluna Tempo de Empresa

Considerações: foi feito um label encoding para a informação poder ser traduzida de maneira mais computacional

In [ ]:
col = "Tempo de Empresa"

exibe_coluna(col)

subs = {
    "Até 6 meses": 0,
    "6 meses a 1 ano": 1,
    "1 a 3 anos": 2,
    "3 a 6 anos": 3,
    "6 a 10 anos": 4,
    "Acima de 10 anos": 5,
    "10 a 15 anos": 5,
    "15 a 20 anos": 5,
    "20 a 25 anos": 5,
    "30 a 35 anos": 5,
}

df[col].replace(subs, inplace=True)

exibe_coluna(col)

## Coluna Dia da Semana

Considerações: foi padronizado a maneira de escrita do dia da semana, além da formatação padrão sem acentos e tudo minúsculo

In [ ]:
col = "Dia da Semana"

exibe_coluna(col)

df[col] = df[col].apply(formata_string)
df[col] = df[col].apply(remove_espaços)

exibe_coluna(col)

## Coluna Dia

Considerações: não foram necessárias alterações; foi utilizada para criar a coluna **Data**

In [ ]:
col = "Dia"
exibe_coluna(col)

## Coluna Mês

Considerações: não foram necessárias alterações; foi utilizada para criar a coluna **Data**

In [ ]:
col = "Mês"
exibe_coluna(col)

## Coluna Ano

Considerações: não são necessárias alterações.; foi utilizada para criar a coluna **Data**

In [ ]:
col = "Ano"
exibe_coluna(col)

## Coluna Hora

Considerações: Foi necessário substituir os horários para ser lido de maneira uniforme.

**específico:**
    Foram trocados os horários de início e fim apenas pelo horário de início, sem minutos.

In [ ]:
col = "Hora"
exibe_coluna(col)

In [ ]:
valores = df["Hora"].unique()

for hora in valores:
    try:
        subs[hora] = hora[:2]
    except:
        # subs['Hora'][np.nan] = 'nao informado'
        pass

print(json.dumps(subs, indent=4))

df["Hora"].replace(subs, inplace=True)

exibe_coluna(col)

## Criando a coluna **data** 
Considerações: Para ter mais fácil manipulação  dos dados, é adicionada a coluna 'Data' com o tipo datetime.

In [ ]:
print(df["Dia"].unique())
print(df["Mês"].unique())
print(df["Ano"].unique())

df["year"] = df["Ano"]
df["month"] = df["Mês"]
df["day"] = df["Dia"]
df["hour"] = df["Hora"].fillna(
    12
    # só tem uma linha com NaN, decidi preencher com um valor para não perder as outras informações.
)
df["hour"] = df["hour"].astype(int)
df["Data"] = pd.to_datetime(df[["year", "month", "day", "hour"]])

remove_coluna(df, "year")
remove_coluna(df, "month")
remove_coluna(df, "day")
remove_coluna(df, "hour")

exibe_coluna("Data")

## Coluna Descrição

Considerações: foi retirado acentos, maiúsculas, cedilhas e espaço.

In [ ]:
col = "Descrição"
df["Descrição"] = df["Descrição"].astype(dtype="string")
print(df["Descrição"].shape)
df["Descrição"].isna().reset_index().query("Descrição==True")[
    "index"
]  # acha o nan na tabela.

# Padronizando a coluna
df["Descrição"] = df["Descrição"].apply(formata_string)
df["Descrição"] = df["Descrição"].apply(formata_string)

# Troca todos os ';' por ','
df["Descrição"].replace(";", ",", inplace=True)
df["Descrição"]

## Coluna Local

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
col = "Local"
exibe_coluna(col)

df["Local"] = df["Local"].apply(formata_string)
print(df["Local"].unique())
df["Local"]

## Coluna Tipo de Trabalho

Considerações: Foi ajustada uma única entrada errônea como NaN; fora esta não foram feitas alterações e foi retirada acentos, espaços e letras maiúsculas

In [ ]:
col = "Tipo de Trabalho"
df[col] = df[col].astype(dtype="string")

# exibe_coluna(col)

df[col].replace(to_replace="Sem Lesões", value=np.nan, inplace=True)
df[col] = df[col].apply(formata_string)
df[col]

## Coluna Município

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
col = "Município"
df[col].shape
# df["Município"] = df["Município"].astype(dtype="string")

# df[col] = df[col].apply(formata_string)

exibe_coluna(col)

## Coluna Estado
considerações: não são necessárias alterações.

In [ ]:
col = "Estado"
exibe_coluna(col)

## Coluna Instalação 
Considerações: Remoção de acentos e formatação para letra minúscula. Também foi feito análise de valores únicos para conferir se há erros de digitação.

In [ ]:
col = "Instalação"
exibe_coluna(col)

df[col] = df[col].apply(formata_string)

exibe_coluna(col)

In [ ]:
# Garantir que todos os valores na coluna são strings
df["Instalação"] = df["Instalação"].astype(str).fillna("")

# Encontrar valores similares na coluna 'Instalação'
similar_values = achar_valores_similares(df["Instalação"])
tam = len(similar_values)
i = 0
while i < tam:
    if similar_values[i][2] > 92:
        print(similar_values[i])
        print("\n")
    i += 1

In [ ]:
col = "Instalação"
df[col] = df[col].replace("se araraqura", "se araraquara", regex=True)
df[col] = df[col].replace("empresa - sede", "sede empresa", regex=True)
df[col] = df[col].replace("usina de sobradinho",
                          "usina sobradinho", regex=True)
df[col] = df[col].replace("usina de boa esperanca",
                          "usina boa esperanca", regex=True)
df[col] = df[col].replace("", "", regex=True)
# antigo: 287
exibe_coluna(col)

## Colunas Latitude e Longitude
considerações: remove "°" e troca ',' por '.' nas coordenadas e transforma todos os valores para float.

In [ ]:
col = "Latitude"
exibe_coluna(col)


col = "Longitude"
exibe_coluna(col)

In [ ]:
# Remove "°" das coordenadas e garante que todas são float
pd.set_option("display.max_columns", None)

# Aplicar a função às colunas 'Latitude' e 'Longitude'
df["Latitude"] = df["Latitude"].apply(processa_coordenada)
df["Longitude"] = df["Longitude"].apply(processa_coordenada)
df["Latitude"] = df["Latitude"].apply(processa_coordenada)
df["Longitude"] = df["Longitude"].apply(processa_coordenada)

# Exibir as primeiras 10 linhas
print(df.head(10))

## Coluna Vice-presidência
considerações: não foram feitas alterações. 


In [ ]:
col = "Vice-presidência"
exibe_coluna(col)

## Coluna Diretoria 

considerações: Foi ajustada apenas uma linha errônea, que estava com capitalização errônea. Também foi padronizado para letra minúscula.

In [ ]:
col = "Diretoria"
# exibe_coluna(col)
df[col] = df[col].astype(str)

subs = {
    "Engenharia": "engenharia",
    "Operações": "operações",
    "Outros": "outros",
    "VSS": "vss",
}

df[col].replace(subs, inplace=True)

exibe_coluna(col)

## Coluna Gerência Executiva 
Considerações: Remoção de acentos e formatação para letra minúscula. Também foi feito análise de valores únicos para conferir se há erros de digitação.

In [ ]:
col = "Gerência Executiva"
exibe_coluna(col)

df[col] = df[col].apply(formata_string)

exibe_coluna(col)

df[col].unique()

In [ ]:
# Garantir que todos os valores na coluna são strings
df["Gerência Executiva"] = df["Gerência Executiva"].astype(str).fillna("")

# Encontrar valores similares na coluna 'Gerência Executiva'
similar_values = achar_valores_similares(df["Gerência Executiva"])
tam = len(similar_values)
i = 0
while i < tam:
    print(similar_values[i])
    print("\n")
    i += 1
print(similar_values)

In [ ]:
# Substituir usando expressões regulares
col = "Gerência Executiva"
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "dgp departramento de gestao de pessoas",
    "departamento de gestao de pessoas",
    regex=True,
)
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "fr.sdepartamento de produtividade e qualidade da unidade sul",
    "departamento de produtividade e qualidade da unidade sul",
    regex=True,
)
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "eeig - implantacao de geracao", "eeig - implantacao da geracao", regex=True
)
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "superintedencia de gestao de pessoas",
    "superintendencia de gestao de pessoas",
    regex=True,
)
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "oog.s - producao de transmissa", "oog.s - producao de transmissao", regex=True
)
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "superintendencia de producao da geracao hidraulica\nproducao da geracao hidraulica",
    "superintendencia de producao da geracao hidraulica",
    regex=True,
)
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "deg departamento engenharia de geracao",
    "departamento de engenharia de geracao",
    regex=True,
)
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "engenharia de geracao", "departamento de engenharia de geracao", regex=True
)
df["Gerência Executiva"] = df["Gerência Executiva"].replace(
    "dmo departamento de operacao e apoio a  manutencao",
    "departamento de manutencao e apoio a operacao",
    regex=True,
)

df["Gerência Executiva"].unique().shape
exibe_coluna(col)

## Coluna Gerência 

Considerações: Remoção de acentos e formatação para letra minúscula. Também foi feito análise de valores únicos para conferir se há erros de digitação.

In [ ]:
col = "Gerência"
exibe_coluna(col)

df[col] = df[col].apply(formata_string)

exibe_coluna(col)

In [ ]:
# Garantir que todos os valores na coluna são strings
df["Gerência"] = df["Gerência"].astype(str).fillna("")

valores_similares_gerencia = achar_valores_similares(df["Gerência"])
tam = len(valores_similares_gerencia)
i = 0
while i < tam:
    if valores_similares_gerencia[i][2] > 95:
        print(valores_similares_gerencia[i])
        print("\n")

    i += 1

In [ ]:
col = "Gerência"
df[col] = df[col].replace(
    "departamento de saude, seguranca e bem-estar no trabalho",
    "departamento de saude, seguranca e bem-estar do trabalho",
    regex=True,
)  # isso nao esta fazendo nada
df[col] = df[col].replace(
    "departamento de operacao regional paulo afonso",
    "departamento de operacao regional de paulo afonso",
    regex=True,
)
df[col] = df[col].replace(
    "departamento de operacao regional teresina",
    "departamento de operacao regional de teresina",
    regex=True,
)
df[col] = df[col].replace(
    "departamento de operacoes regional de fortaleza",
    "departamento de operacao regional de fortaleza",
    regex=True,
)
df[col] = df[col].replace(
    "departamento de infraestrurura e servicos gerais do csc",
    "departamento de infraestrutura e servicos gerais do csc",
    regex=True,
)
df[col] = df[col].replace(
    "departamento de infraestrutura de servicos gerais csc",
    "departamento de infraestrutura e servicos gerais do csc",
    regex=True,
)  # coloquei essa como certo porque havia outras escritas dessa forma mesmo que com erro de digitacao
df[col] = df[col].replace("des,o", "des.o", regex=True)
df[col] = df[col].replace("drt,o", "drt.o", regex=True)
df[col] = df[col].replace("dit,e", "dit.e", regex=True)
df[col] = df[col].replace(
    "ooto.s  regional de manutencao do oeste",
    "ooto.s regional de manutencao do oeste",
    regex=True,
)
df[col] = df[col].replace(
    "departamento de operacoes regional de fortaleza",
    "departamento de operacao regional de fortaleza",
    regex=True,
)
df[col] = df[col].replace(
    "ooss,c - operacao regional de salvador",
    "ooss.c - operacao regional de salvador",
    regex=True,
)
df[col] = df[col].replace(
    "oonf,c - operacao regional de fortaleza",
    "oonf.c - operacao regional de fortaleza",
    regex=True,
)
df[col] = df[col].replace(
    "departamento regional de producao e trasmissao do acre e rondonia",
    "departamento regional de producao e transmissao do acre e rondonia",
    regex=True,
)

# antiga: 220 entradas diferentes, agora: 208
exibe_coluna(col)

## Coluna 'Divisão'

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Divisão"] = df["Divisão"].apply(formata_string)
df["Divisão"] = df["Divisão"].apply(formata_string)

print("Tipos da coluna Divisão:")
checa_tipos(df["Divisão"])

## Colunas 'Data Início do Afastamento' e 'Data Fim do Afastamento'

Considerações: Foi feita conversão de datas no formato excel para datetime

In [ ]:
df["Data Início do Afastamento"] = df["Data Início do Afastamento"].apply(
    converte_data_excel
)
df["Data Início do Afastamento"] = df["Data Início do Afastamento"].replace(
    " ", np.nan)
df["Data Fim do Afastamento"] = df["Data Fim do Afastamento"].apply(
    converte_data_excel)

print("Tipos da coluna Data Início do Afastamento:")
checa_tipos(df["Data Início do Afastamento"])

print("\nTipos da coluna Data Fim do Afastamento:")
checa_tipos(df["Data Fim do Afastamento"])

## Coluna 'Dias Perdidos'

Considerações: Substituição de hífen por NaN

In [ ]:
# Substitui '-' por NaN
df["Dias Perdidos"] = df["Dias Perdidos"].replace("-", np.nan)

print("Tipos da coluna Dias Perdidos:")
checa_tipos(df["Dias Perdidos"])

## Coluna 'Dias Debitados'

Considerações: Substituição de hífen e string 'Não' por NaN

In [ ]:
# Substitui '-' e 'Não' por NaN
df["Dias Debitados"] = df["Dias Debitados"].replace("-", np.nan)
df["Dias Debitados"] = df["Dias Debitados"].replace("Não", np.nan)

print("Tipos da coluna Dias Debitados:")
checa_tipos(df["Dias Debitados"])

## Coluna 'Reabilitado'

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Reabilitado"] = df["Reabilitado"].apply(formata_string)
df["Reabilitado"] = df["Reabilitado"].apply(formata_string)

print("Tipos da coluna Reabilitado:")
checa_tipos(df["Reabilitado"])
exibe_coluna("Reabilitado")

## Coluna 'Dias Perdidos Corrigidos'

Considerações: Substituição de hífens por NaN

In [ ]:
# Substitui '-' por NaN
df["Dias Perdidos Corrigidos"] = df["Dias Perdidos Corrigidos"].replace(
    "-", np.nan)

print("Tipos da coluna Dias Perdidos Corrigidos:")
checa_tipos(df["Dias Perdidos Corrigidos"])
exibe_coluna("Dias Perdidos Corrigidos")

## Coluna 'Data da CAT'
 Considerações: Substituição de hífens e data inválida por NaN

In [ ]:
# Substitui '-'por NaN
df["Data da CAT"] = df["Data da CAT"].replace("-", np.nan)

# Substitui '19/91/2024' (data inexistente) por NaN
df["Data da CAT"] = df["Data da CAT"].replace("19/91/2024", np.nan)

print("Tipos da coluna Data da CAT:")
checa_tipos(df["Data da CAT"])

## Coluna 'N° da CAT'

Considerações: Conversão de ints em strings.

In [ ]:
# Converter todas as células da coluna n° da cat que são ints para string usando iterrows
for index, row in df.iterrows():
    if type(row["N° da CAT"]) == int:
        df.at[index, "N° da CAT"] = str(row["N° da CAT"])

print("Tipos da coluna N° da CAT:")
checa_tipos(df["N° da CAT"])

## Coluna 'Agente Causador'

Considerações: Remoção de acentos e formatação para letra minúscula, remoção de códigos avulsos.

In [ ]:
df["Agente Causador"] = df["Agente Causador"].apply(formata_string)
df["Agente Causador"] = df["Agente Causador"].apply(formata_string)

print("Tipos da coluna Agente Causador:")
checa_tipos(df["Agente Causador"])

# Remove strings que comecam com um numero e hifen com regex, exemplo: 303010900 - ferramenta manual sem forca motriz.
df["Agente Causador"] = df["Agente Causador"].str.replace(
    r"^\d+ - ", "", regex=True)
exibe_coluna("Agente Causador")

## Coluna 'Tipo de Lesão'

Considerações: Remoção de acentos e formatação para letra minúscula.

In [ ]:
df["Tipo de Lesão"] = df["Tipo de Lesão"].apply(formata_string)
df["Tipo de Lesão"] = df["Tipo de Lesão"].apply(formata_string)

print("Tipos da coluna Tipo de Lesão:")
checa_tipos(df["Tipo de Lesão"])

df["Tipo de Lesão"] = df["Tipo de Lesão"].str.replace(
    r"^\d+ - ", "", regex=True)

df["Tipo de Lesão"] = df["Tipo de Lesão"].str.replace(
    r"^\d+ - ", "", regex=True)
exibe_coluna("Tipo de Lesão")

## Coluna 'Parte do Corpo Atingida'

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Parte do Corpo Atingida"] = df["Parte do Corpo Atingida"].apply(
    formata_string)
df["Parte do Corpo Atingida"] = df["Parte do Corpo Atingida"].apply(
    formata_string)

print("Tipos da coluna Parte do Corpo Atingida:")
checa_tipos(df["Parte do Corpo Atingida"])

df["Parte do Corpo Atingida"] = df["Parte do Corpo Atingida"].str.replace(
    r"^\d+ - ", "", regex=True
)


df["Parte do Corpo Atingida"] = df["Parte do Corpo Atingida"].str.replace(
    r"^\d+ - ", "", regex=True
)

exibe_coluna("Parte do Corpo Atingida")

## Coluna 'Gravidade'

Considerações: Remoção de acentos e formatação para letra minúscula

In [ ]:
df["Gravidade"] = df["Gravidade"].apply(formata_string)
df["Gravidade"] = df["Gravidade"].apply(formata_string)

print("Tipos da coluna Gravidade:")
checa_tipos(df["Gravidade"])

## Coluna 'Potencial'
Considerações: Sem alterações.

In [ ]:
print("Tipos da coluna Potencial:")
checa_tipos(df["Potencial"])

## Coluna Grau de Risco

Considerações: foi feito um label encoding para a informação poder ser traduzida de maneira mais computacional

In [ ]:
print(df["Grau de Risco"].value_counts(dropna=False))
# como temos três strings de escala, podemos aplicar ordinal encoding para substituir por números
# 1 - baixo, 2 - médio, 3 - alto

subs = {
    "Baixo": 1,
    "Médio": 2,
    "Alto": 3,
}

df["Grau de Risco"].replace(subs, inplace=True)

print(df["Grau de Risco"].value_counts(dropna=False))

## Coluna Plano de ação

Considerações: foi feito um label encoding para a informação poder ser traduzida de maneira mais computacional

In [ ]:
print(df["Plano de ação"].value_counts(dropna=False))
# temos três classificações realmente únicas na coluna: concluído, pendente e em andamento
# utilizamos label encoding para atribuir um numero unico para cada
# 1 - pendente, 2 - em andamento, 3 - concluido

subs = {
    "Pendente": 1,
    "Em andamento": 2,
    "Concluído": 3,
    "SIM": 3,
    "Concluido": 3,
}

# df["Plano de ação"].replace(subs["Plano de ação"],inplace=True)
df["Plano de ação"].replace(subs, inplace=True)

print(df["Plano de ação"].value_counts(dropna=False))

## Coluna Relatório de Investigação

Considerações: Foi feito um label encoding para a informação poder ser traduzida de maneira mais computacional.

In [ ]:
# 0 - não enviado, 1 - concluido, 2 - pendente, 3 - em investigação

subs = {
    "Não": 0,
    "Concluído": 1,
    "Concluido": 1,
    "SIM": 1,
    "Sim": 1,
    "Pendente": 2,
    "Em Andamento": 3,
    "Em andamento": 3,
    "Em investigação": 3,
}

df["Relatório de Investigação"].replace(subs, inplace=True)

print(df["Relatório de Investigação"].value_counts(dropna=False))

## Coluna Cartão Alerta

Considerações: foi feito um label encoding para a informação poder ser traduzida de maneira mais computacional, além de criar uma nova coluna com dados mais detalhados para os casos onde esta era uma solução mais apropriada.

In [ ]:
print(df["Cartão Alerta"].value_counts(dropna=False))
# 0 - não enviado, 1 - enviado, 2 - em revisão

subs = {
    "Não enviado": 0,
    "SIM": 1,
    "Sim": 1,
    "Em revisão": 2,
}

df["Cartão Alerta"].replace(subs, inplace=True)
# transformamos texto descritivo em nova coluna
df["Cartão Alerta"] = df["Cartão Alerta"].apply(
    lambda x: 1 if isinstance(x, str) else x
)
# print(df["Cartão Alerta"].value_counts(dropna=False))

## Coluna Compromissos (Regra de Ouro)

Considerações: foi feito um label encoding utilizando diretamente a numeração aplicada já na tabela.

In [ ]:
print(df["Compromissos (Regra de Ouro)"].value_counts(dropna=False))
# 0 - Outros, 1 - Álcool e Drogas, 2 - Percepção de Risco, 3 - Trabalho em Altura, 4 - Eletricidade, 5 - Espaço Confinado, 6 - Linha de Perigo
# 7 - Bloqueio e Proteção, 8 - Veículos e Equipamentos Móveis, 9 - Trabalho a Quente, 10 - Produtos Químicos Perigosos, 11 - Não se Aplica
df["Compromissos (Regra de Ouro)"].replace(
    "Outros", "0 - Outros", inplace=True)
df["Compromissos (Regra de Ouro)"] = df["Compromissos (Regra de Ouro)"].apply(
    lambda x: int(x[:2]) if isinstance(x, str) else np.nan
)

print(df["Compromissos (Regra de Ouro)"].value_counts(dropna=False))

## Colunas Causa raiz 1-8

Considerações: feita limpeza simples adicionando NaN no lugar de strings vazias ou coisas do tipo.

In [ ]:
print("\nAntes de limpar:\n\n")
for elem in df["Causa raiz 1"].unique():
    print(elem)

df["Causa raiz 1"].replace("-", np.nan, inplace=True)
df["Causa raiz 1"] = df["Causa raiz 1"].apply(
    lambda x: unidecode(x.lower().strip(
        "\n")) if isinstance(x, str) else np.nan
)  # adicionando NaN no lugar de vazio

print("\nDepois de limpar:\n\n")
for elem in df["Causa raiz 1"].unique():
    print(elem)

In [ ]:
print("\nAntes de limpar:\n\n")
for elem in df["Causa raiz 2"].unique():
    print(elem)


df["Causa raiz 2"].replace("-", np.nan, inplace=True)
df["Causa raiz 2"] = df["Causa raiz 2"].apply(
    lambda x: unidecode(x.lower().strip(
        "\n")) if isinstance(x, str) else np.nan
)

print("\nDepois de limpar:\n\n")
for elem in df["Causa raiz 2"].unique():
    print(elem)

In [ ]:
print("\nAntes de limpar:\n\n")
for elem in df["Causa raiz 3"].unique():
    print(elem)

df["Causa raiz 3"].replace("-", np.nan, inplace=True)
df["Causa raiz 3"] = df["Causa raiz 3"].apply(
    lambda x: unidecode(x.lower().strip(
        "\n")) if isinstance(x, str) else np.nan
)

print("\nDepois de limpar:\n\n")
for elem in df["Causa raiz 3"].unique():
    print(elem)

In [ ]:
print("\nAntes de limpar:\n\n")
for elem in df["Causa raiz 4"].unique():
    print(elem)

df["Causa raiz 4"].replace("-", np.nan, inplace=True)
df["Causa raiz 4"] = df["Causa raiz 4"].apply(
    lambda x: unidecode(x.lower().strip(
        "\n")) if isinstance(x, str) else np.nan
)

print("\nDepois de limpar:\n\n")
for elem in df["Causa raiz 4"].unique():
    print(elem)

In [ ]:
print(df["Causa raiz 5"].value_counts(dropna=False))

df["Causa raiz 5"].replace("-", np.nan, inplace=True)
df["Causa raiz 5"] = df["Causa raiz 5"].apply(
    lambda x: (
        unidecode(x.lower().strip(".").strip("\n")
                  ) if isinstance(x, str) else np.nan
    )
)
print(df["Causa raiz 5"].value_counts(dropna=False))

In [ ]:
print(df["Causa raiz 6"].value_counts(dropna=False))
# como temos hífens representando NaN, trocamos por NaN para padronizar
df["Causa raiz 6"].replace("-", np.nan, inplace=True)
df["Causa raiz 6"].replace(
    "Falha na Supervsão", "Falha na supervisão", inplace=True)
df["Causa raiz 6"].replace(
    "Configuração doo sistema", "Configuração do sistema", inplace=True
)
df["Causa raiz 6"] = df["Causa raiz 6"].apply(
    lambda x: unidecode(x.lower().strip(".")) if isinstance(x, str) else np.nan
)

print(df["Causa raiz 6"].value_counts(dropna=False))

In [ ]:
print(df["Causa raiz 7"].value_counts(dropna=False))
# como temos hífens representando NaN, trocamos por NaN para padronizar
df["Causa raiz 7"].replace("-", np.nan, inplace=True)
df["Causa raiz 7"] = df["Causa raiz 7"].apply(
    lambda x: unidecode(x.lower().strip(".")) if isinstance(x, str) else np.nan
)
subs["Causa raiz 7"] = "- trocado por NaN"

print(df["Causa raiz 7"].value_counts(dropna=False))

In [ ]:
print(df["Causa raiz 8"].value_counts(dropna=False))
# como temos hífens representando NaN, trocamos por NaN para padronizar
df["Causa raiz 8"].replace("-", np.nan, inplace=True)
df["Causa raiz 8"] = df["Causa raiz 8"].apply(
    lambda x: unidecode(x.lower().strip(".")) if isinstance(x, str) else np.nan
)
subs["Causa raiz 8"] = "- trocado por NaN"

print(df["Causa raiz 8"].value_counts(dropna=False))

## Coluna Chave

In [ ]:
# Descobrindo se alguma coluna só tem NaN e removendo do dataframe
for name in df:
    if df[name].isna().sum() == len(df[name]):
        print("Coluna %s removida" % (name))
        remove_coluna(df, name)

A coluna chave foi removida porque só tinha valores NaN.

## Coluna Local de instalação
Considerações: Pelas coordenadas, pegaremos os locais de instalação mais perto daquela coordenada calculando com a fórmula de haversine.

In [ ]:
# Definindo os locais de instalacao
df = calcula_local_instalacao_acidente(
    50, df, dataFrameInstalacao  # raio arbitrario de 50 km
)
df["Local de instalação"]

Como inicialmente estamos fazendo so para Eletrosul, temos que checar se alguma outra empresa esta tendo um local de instalacao erroneamente. Futuramente, quando usaremos dados de todas as empresa, esse passo sera desnecessario.

In [ ]:
checar_acidente_fora_eletrosul(df)

## Conversão de dataframe pandas para csv

Considerações: Para colocar em csv, primeiramente temos que decidir um caracter para separar as colunas, e para decidir tal caracter nao pode estar um alguma das linhas.

In [ ]:
# Checar se o DataFrame contém o caractere '|'
if possui_pipe(df):
    print("O DataFrame contém o caractere '|'.")
else:
    print("O DataFrame pode ser salvo em csv.")

# Exibir DataFrame original
print("DataFrame original:")
print(df)

In [ ]:
df.to_csv(
    pathUtil + "acidentes/" + "BD_Acidentes_Padronizado.csv",
    encoding="utf-8",
    index=False,
    sep="|",
)